## GenImage
In this use case it is shown how to extract information from a PDF file through LLM queries with RAG (Retrieval Augmented Generation) technology. For this use case is necessary the use of a vector database (in this case FAISS), embeddings and OpenAI model calls. To show the final result, the model is embedded on a Gradio UI.

In [1]:
from dotenv import load_dotenv
import os

from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

load_dotenv("apis.env")
# hf_api_key = os.environ['HF_API_KEY']

True

In [1]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_id = "stabilityai/stable-diffusion-2-1"

# Use the DPMSolverMultistepScheduler (DPM-Solver++) scheduler here instead
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]
    
image.save("astronaut_rides_horse.png")


c:\Users\Nacho\anaconda3\envs\TFM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]



















































In [1]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]  
    
image.save("astronaut_rides_horse.png")

c:\Users\Nacho\anaconda3\envs\TFM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...:  29%|██▊       | 2/7 [00:02<00:06,  1.33s/it]`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
Loading pipeline components...: 100%|██████████| 7/7 [00:32<00:00,  4.65s/it]


AssertionError: Torch not compiled with CUDA enabled

In [1]:
from langchain.chains import Chain
from langchain.llms import HuggingfaceEndpoint
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize the Hugging Face Model and Tokenizer
model_name = "your-model-name"  # Replace with the actual model name from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up the LangChain wrapper
hf_endpoint = HuggingfaceEndpoint(model, tokenizer)

# Create a chain for image generation
image_generation_chain = Chain([hf_endpoint])

# Example input for image generation
input_text = "Describe your image generation prompt here"

# Generate an image
output = image_generation_chain.run(input_text)

# Output the result
print(output)


ImportError: cannot import name 'Chain' from 'langchain.chains' (c:\Users\Nacho\anaconda3\envs\TFM\Lib\site-packages\langchain\chains\__init__.py)

# Advanced Gradio app

In [2]:
import gradio as gr

# CSS Template 
theme = gr.themes.Base(
    primary_hue="fuchsia",
).set(
    body_background_fill='*neutral_50',
    body_text_color='*neutral_500',
    body_text_weight='300',
    background_fill_primary='*neutral_50',
    background_fill_secondary='*primary_50',
    border_color_primary='*primary_400',
    color_accent_soft='*primary_300',
    link_text_color='*primary_300',
    link_text_color_active='*neutral_300',
    link_text_color_hover='*primary_100',
    link_text_color_visited='*neutral_400',
    code_background_fill='*primary_200',
    button_secondary_background_fill='*neutral_100',
    button_secondary_border_color='*neutral_900',
    button_secondary_text_color='*primary_400',
    button_cancel_background_fill='*primary_600',
    button_cancel_background_fill_hover='*primary_700',
    button_cancel_text_color='*neutral_50',
    slider_color='*primary_500'
)


In [3]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

def extract_sentence(dictionary):
    # Extract the value associated with the key 'result'
    sentence = dictionary.get('result', '')
    return sentence

def model_hyperparameters(temperature, max_tokens, db):#
    llm = OpenAI(
        model_name="gpt-3.5-turbo-instruct",
        temperature=temperature,
        max_tokens=max_tokens,
        streaming=True
        )
    chain = RetrievalQA.from_llm(llm=llm, retriever=db.as_retriever())
    return chain

def respond(message, output_label, temperature=0.7, max_tokens=32):
    db = pdf_vectorized_loader(output_label)
    prompt = message
    chain = model_hyperparameters(temperature, max_tokens, db)
    completion = chain(prompt, return_only_outputs=True)
    return extract_sentence(completion)

def pdf_vectorized_loader(pdf_file):
    loader = PyPDFLoader(pdf_file)
    pages = loader.load_and_split()
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(pages, embeddings)
    return db

def process_file(uploaded_file):
    if uploaded_file is not None:
        filename = uploaded_file.name
        return filename
    return "No file uploaded."

with gr.Blocks(theme=theme) as demo: 
    gr.Markdown("# TFM RAG App")
    gr.Markdown("With this RAG app, you can generate augmented responses from a selected PDF file.")

    file_input = gr.File(file_types=[".pdf"])
    output_label = gr.Textbox(visible=False)

    file_input.change(process_file, inputs=file_input, outputs=output_label)

    msg = gr.Textbox(label="Ask a question")
    with gr.Accordion(label="Advanced options",open=False):
        temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1.0, value=0.2, step=0.1, info="Regulates the creativity of the answers")
        max_tokens = gr.Slider(label="Max tokens", value=64, maximum=256, minimum=8, step=1, info="Regulates the length of the answers")
    completion = gr.Textbox(label="Response")
    btn = gr.Button("Submit", variant="primary")
    clear = gr.ClearButton(components=[msg, completion], value="Clear console", variant="stop")
    
    btn.click(respond, inputs=[msg, output_label, temperature, max_tokens], outputs=[completion])
    msg.submit(respond, inputs=[msg, output_label, temperature, max_tokens], outputs=[completion])

    gr.Markdown("Created by Ignacio Ojeda Sánchez (www.linkedin.com/in/ignacio-ojeda-sánchez-610924225)", header_links=True)

gr.close_all()
demo.queue().launch(share=True)    

Running on local URL:  http://127.0.0.1:7904
Running on public URL: https://3eeaf86a709358349c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [47]:
gr.close_all()

Closing server running on port: 7890
